In [11]:
import pandas as pd

# 파일 경로 설정
temperature_accum_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울 적산온도 데이터.csv'
temperature_analysis_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울시 기온분석.csv'
wind_analysis_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울시 바람 분석.csv'
humidity_analysis_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울시 습도 분석.csv'
sunshine_solar_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울시 일조일사 데이터.csv'
precipitation_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/서울시 강수량 데이터.csv'

# 각 파일을 cp949 인코딩과 적절한 skiprows 옵션을 사용하여 로드
temperature_accum = pd.read_csv(temperature_accum_path, encoding='cp949', skiprows=3)
temperature_analysis = pd.read_csv(temperature_analysis_path, encoding='cp949', skiprows=7)
wind_analysis = pd.read_csv(wind_analysis_path, encoding='cp949', skiprows=15)
humidity_analysis = pd.read_csv(humidity_analysis_path, encoding='cp949', skiprows=16)
sunshine_solar = pd.read_csv(sunshine_solar_path, encoding='cp949', skiprows=17)
precipitation = pd.read_csv(precipitation_path, encoding='cp949',skiprows=7)

# 필요한 경우, 열 이름 변경 및 날짜 형식 변환
temperature_accum.rename(columns={'년도': '날짜'}, inplace=True)
temperature_accum['날짜'] = pd.to_datetime(temperature_accum['날짜'], format='%Y')

temperature_analysis['날짜'] = pd.to_datetime(temperature_analysis['날짜'].str.strip(), format='%Y-%m-%d')

# wind_analysis 등 다른 데이터 프레임에도 같은 방식으로 적용
# '일시' 또는 다른 열 이름이 날짜로 사용된 경우 해당 열 이름을 '날짜'로 변경
wind_analysis.rename(columns={'일시': '날짜'}, inplace=True)
wind_analysis['날짜'] = pd.to_datetime(wind_analysis['날짜'].str.strip(), format='%Y-%m-%d')

humidity_analysis.rename(columns={'일시': '날짜'}, inplace=True)
humidity_analysis['날짜'] = pd.to_datetime(humidity_analysis['날짜'].str.strip(), format='%Y-%m-%d')

sunshine_solar.rename(columns={'일시': '날짜'}, inplace=True)
sunshine_solar['날짜'] = pd.to_datetime(sunshine_solar['날짜'].str.strip(), format='%Y-%m-%d')

precipitation['날짜'] = pd.to_datetime(precipitation['날짜'].str.strip(), format='%Y-%m-%d')


In [12]:
# 모든 데이터 프레임의 '날짜' 열을 기준으로 병합
merged_data = pd.merge(temperature_accum, temperature_analysis, on='날짜', how='outer')
merged_data = pd.merge(merged_data, wind_analysis, on='날짜', how='outer')
merged_data = pd.merge(merged_data, humidity_analysis, on='날짜', how='outer')
merged_data = pd.merge(merged_data, sunshine_solar, on='날짜', how='outer')
merged_data = pd.merge(merged_data, precipitation, on='날짜', how='outer')

# 병합된 데이터 프레임을 CSV 파일로 저장
merged_data.to_csv('C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/merged_data.csv', index=False, encoding='cp949')

### 결측치 확인 및 제거 

In [13]:
merged_data.isnull().sum()

날짜                   0
일수                8870
값                 8870
지점_x                 0
평균기온(℃)              0
최저기온(℃)              1
최고기온(℃)              1
\t\t지점번호_x           0
지점명_x                0
평균풍속(m/s)            8
최대풍속(m/s)            6
최대풍속풍향(deg)          6
\t최대풍속시각             6
최대순간풍속(m/s)          6
최대순간풍속풍향(deg)        6
\t최대순간풍속시각           6
\t\t지점번호_y           0
지점명_y                0
평균습도(%rh)            0
최저습도(%rh)            1
\t\t지점번호             0
지점명                  0
일조합(hr)             17
일조율(%)               1
일사합(MJ/m2)          31
지점_y                 0
강수량(mm)          11617
dtype: int64

강수량의 결측치는 0으로 대체

In [14]:
merged_data['강수량(mm)'] = merged_data['강수량(mm)'].fillna(0)

In [15]:
# 수치형 열에 대해 결측치를 앞뒤 10개 값의 평균으로 대체
for column in merged_data.select_dtypes(include=['float64', 'int64']).columns:
    # 해당 열의 결측치가 아닌 값들에 대해 21개의 윈도우 크기로 롤링 평균을 계산
    rolled_series = merged_data[column].rolling(window=21, min_periods=1, center=True).mean()
    
    # 결측치만을 대상으로 롤링 평균 결과로 대체
    merged_data[column] = merged_data[column].fillna(rolled_series)

### 불필요한 칼럼은 제외

In [17]:
merged_data.drop(columns=['일수','값','지점명_x','지점명_y','지점명','\t\t지점번호' ,'\t최대풍속시각','\t\t지점번호_y','\t\t지점번호','\t최대순간풍속시각','지점_y','지점_x'] ,inplace=True)

In [22]:
merged_data.isnull().sum()

날짜               0
평균기온(℃)          0
최저기온(℃)          0
최고기온(℃)          0
\t\t지점번호_x       0
평균풍속(m/s)        0
최대풍속(m/s)        0
최대풍속풍향(deg)      0
최대순간풍속(m/s)      0
최대순간풍속풍향(deg)    0
평균습도(%rh)        0
최저습도(%rh)        0
일조합(hr)          0
일조율(%)           0
일사합(MJ/m2)       0
강수량(mm)          0
dtype: int64

### 날짜형으로 변환

In [18]:
merged_data['날짜'] = pd.to_datetime(merged_data['날짜'], format='%Y%m%d')

In [23]:
merged_data

,날짜,평균기온(℃),최저기온(℃),최고기온(℃),\t\t지점번호_x,평균풍속(m/s),최대풍속(m/s),최대풍속풍향(deg),최대순간풍속(m/s),최대순간풍속풍향(deg),평균습도(%rh),최저습도(%rh),일조합(hr),일조율(%),일사합(MJ/m2),강수량(mm)
0,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0
1,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0
2,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0
3,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0
4,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17997,2024-05-04,21.4,14.6,28.5,108,1.9,4.3,250.0,7.7,270.0,44.8,27.0,12.5,89.9,26.61,0.0
17998,2024-05-05,17.8,15.7,20.6,108,2.5,5.6,200.0,10.7,180.0,87.4,44.0,0.0,0.0,2.17,48.3
17999,2024-05-06,13.9,13.4,15.7,108,3.1,4.9,200.0,9.4,200.0,92.9,88.0,0.0,0.0,2.71,12.3
18000,2024-05-07,13.3,12.2,15.0,108,3.0,4.9,70.0,9.5,70.0,77.3,60.0,0.0,0.0,4.75,2.4


In [19]:
# 전치리 완료된 데이터 프레임을 CSV 파일로 저장
merged_data.to_csv('C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/merged_data.csv', index=False, encoding='cp949')